In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import fbeta_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
warnings.simplefilter(action='ignore', category=FutureWarning)
import xgboost
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import confusion_matrix

from sklearn import metrics
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import GroupShuffleSplit
import pickle
from sklearn.inspection import permutation_importance
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#os.chdir('../data')


X = pd.read_csv('X_3specialties_equalWeight_subsample.zip',compression='zip', index_col=False)
y = pd.read_csv('y_3specialties_equalWeight_subsample.zip',compression='zip')
groups = pd.read_csv('groups_3specialties_equalWeight_subsample.zip',compression='zip')

X = X.iloc[:,1:]
y = y.iloc[:,1:]
groups = groups.iloc[:,1:]

y_columns = y.columns

#le = LabelEncoder()
#y = y.values.ravel()
#y = le.fit_transform(y)
#y = pd.DataFrame(y)
#y.columns = y_columns

In [ ]:
def ML_RFpipeline_kfold(X, y, groups, random_state,n_folds):
    # create a test set
    
    splitter = GroupShuffleSplit(n_splits=1,test_size=0.2,random_state=random_state)
    
    for i_other,i_test in splitter.split(X, y, groups):
        X_other, y_other, groups_other = X.iloc[i_other], y.iloc[i_other], groups.iloc[i_other]
        X_test, y_test, groups_test = X.iloc[i_test], y.iloc[i_test], groups.iloc[i_test]
        
    kf = GroupKFold(n_splits=n_folds)
    
    # create the pipeline: preprocessor + supervised ML method
    
    categorical_ftrs = ['Prscrbr_City','Prscrbr_State_Abrvtn','Brnd_Name','Gnrc_Name']

    std_ftrs = ['Tot_Clms',  'Tot_30day_Fills', 'Tot_Day_Suply', 'Tot_Drug_Cst', 
                'Tot_Benes', 'GE65_Tot_Clms', 'GE65_Tot_30day_Fills', 'GE65_Tot_Drug_Cst',
                'GE65_Tot_Day_Suply', 'GE65_Tot_Benes']

    
    numeric_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(estimator = LinearRegression(), 
                                    random_state=random_state,max_iter=50)),
    ('scaler', StandardScaler())])
    
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(sparse=False,handle_unknown='ignore'))])
    
    preprocessor = ColumnTransformer(
        transformers=[
        ('num', numeric_transformer, std_ftrs),
        ('onehot', categorical_transformer, categorical_ftrs)],
        remainder='passthrough')


    pipe = make_pipeline(preprocessor,RandomForestClassifier())
    
 
    
    # the parameter(s) we want to tune
    
    
    param_grid = {'randomforestclassifier__max_depth': [1, 3, 10, 30,100, 300],
                  'randomforestclassifier__min_samples_split': [16, 32, 64, 128],
                  'randomforestclassifier__n_estimators': [1, 3, 10, 30, 100],
                  'randomforestclassifier__random_state':[random_state]
                 }
    
    
    #f05_scorer = make_scorer(fbeta_score, beta=0.5, average = 'macro')
    # prepare gridsearch
    grid = GridSearchCV(pipe, 
                        param_grid=param_grid,
                        scoring = 'accuracy',
                        cv=kf, 
                        return_train_score = True, 
                        n_jobs=4, 
                        verbose=10)
    
    # do kfold CV on _other
    
    grid_result = grid.fit(X_other, y_other.values.ravel(), groups=groups_other)
    
    
    
    #estimator = grid.best_estimator_
    #print "The features are:", 
    
    
    feature_names = std_ftrs + list(grid.best_estimator_[0].named_transformers_['onehot'][0].get_feature_names(categorical_ftrs))
    
   # grid.best_estimator_.get_feature_names_out()
    
    print()
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    
    print(f'Best params: {grid.best_params_}')
    
    #print(f"mean CV: {means} +/ {stds}")
    
    y_test_pred_proba = grid.predict_proba(X_test)
    
    y_test_pred = grid.predict(X_test)
    
    #score = accuracy_score(y_test,y_test_pred)
    
    f_05_score = fbeta_score(y_test, y_test_pred, beta = 0.5, labels=sorted(np.unique(y)), average='macro')
    
    cm = confusion_matrix(y_test, y_test_pred)
    
    accuracy = accuracy_score(y_test, y_test_pred)
    
    best_estimator = grid.best_estimator_
    
    #r = permutation_importance(best_estimator, X_test, y_test, n_repeats=30, random_state=random_state)
    
    print("f05:", f_05_score)
    print("accuracy:", accuracy)
    print()
    return grid, np.array(feature_names), X_test, y_test, f_05_score, cm, accuracy

In [ ]:
%%time
final_models_list = []
test_scores = []
best_params = []
confusion_mat = []
class_met = []
accuracy_scores = []
final_models = []
X_test_set_list =[]
y_test_set_list =[]
featname_list = []

for i in range(5):
    print(f'Random State # {i}')
    print()
    
    fin_grid, featnames, X_test_set, y_test_set, test_score, cmat, accuracy = ML_RFpipeline_kfold(X, y, groups, 42*i , 4)
    
    #featname_list.append(featname)
    
    X_test_set_list.append(X_test_set)
    
    y_test_set_list.append(y_test_set)
    
    final_models_list.append(fin_grid)
    
    test_scores.append(test_score)
    
    confusion_mat.append(cmat)

    accuracy_scores.append(accuracy)
    
    featname_list.append(featnames)

### Save the output for later use

In [ ]:

import pickle

file = open('RF_grid.save', 'wb')

pickle.dump((X_test_set_list, 
             y_test_set_list,
             final_models_list,
             test_scores,
             confusion_mat,
             accuracy_scores,
             featname_list),file)

file.close()

### Random Forest Feature Importance

In [ ]:
file = open('RF_grid.save', 'rb')
X_test, y_test, model, f05, confusionmatrix, accuracy, ftr_names = pickle.load(file)
file.close()


In [ ]:
X_test = X_test[0]
y_test = y_test[0]
grid = model[0]
f05 = f05[0]
confusionmatrix = confusionmatrix[0]
accuracy  = accuracy[0]
ftr_names  = ftr_names[0]

In [ ]:
feature_importances_RF = grid.best_estimator_[1].feature_importances_

len(feature_importances_RF)


In [ ]:
mdi_importances = pd.Series(
    feature_importances_RF, index=ftr_names).sort_values(ascending=True)

mdi_importances_top10 = mdi_importances[-10:]
mdi_importances_top10

In [ ]:
plt.barh(mdi_importances_top10.index, mdi_importances_top10.values)
plt.title("Random Forest Classifier Feature Importances")
plt.savefig("Random Forest Classifier Feature Importances.png", dpi=1200)
plt.show()

In [ ]:
grid.best_estimator_[1]

# Pertubation Feature Importance

In [ ]:

# transform the test set
X_test_transformed = grid.best_estimator_[0].transform(X_test)

print(np.shape(X_test_transformed))


from sklearn.inspection import permutation_importance

with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    r = permutation_importance(grid.best_estimator_[1], X_test_transformed, y_test, 
                               n_repeats=30, random_state=0, scoring = 'accuracy')

In [ ]:
featnames = []
feat_importances_mean = []
feat_std = []


for i in r.importances_mean.argsort()[-5:][::-1]:
    featnames.append(ftr_names[i])
    feat_importances_mean.append(r.importances_mean[i])
    feat_std.append(r.importances_std[i])
    
    
    print(f"{ftr_names[i]:}: {r.importances_mean[i]:.3f}"
          f" +/- {r.importances_std[i]:.3f}")

In [ ]:
file = open('RF_PFI.save', 'wb')

pickle.dump((r),file)

file.close()

In [ ]:
# Visuaize results

sns.set(font_scale=1.2)
plt.figure(figsize=(18, 5))
plt.errorbar(featnames, feat_importances_mean, feat_std, fmt='o',ecolor = 'red',color='black',capsize = 4,)
plt.title('Random Forest Permutation Feature Importance', fontweight = 'bold',fontsize=28)
plt.ylabel("Decrease in model score",fontsize=22)
plt.xlabel("Feature",fontsize=22)
plt.xticks(fontsize=18, rotation=0)
plt.yticks(fontsize=18, rotation=0)
plt.savefig("Random Forest Permutation Feature Importance.png", dpi=1200)
plt.show()

In [ ]:
X_other_transformed = grid.best_estimator_[0].transform(X_other)

In [ ]:
splitter = GroupShuffleSplit(n_splits=1,test_size=0.2,random_state=0)
    
for i_other,i_test in splitter.split(X, y, groups):
    X_other, y_other, groups_other = X.iloc[i_other], y.iloc[i_other], groups.iloc[i_other]
    X_test, y_test, groups_test = X.iloc[i_test], y.iloc[i_test], groups.iloc[i_test]

In [ ]:
#%% time
import shap
shap.initjs() 

# explainer object with RF model
explainer = shap.TreeExplainer(grid.best_estimator_[1], data = X_other_transformed, model_output='probability')

# shape of test set
print(np.shape(X_test_transformed))

# calculate shap values on test set
shap_values = explainer.shap_values(X_test_transformed)

#shape of shape values
print(np.shape(shap_values))


# summary plot
#shap.summary_plot(shap_values, X_test_transformed, feature_names = ftr_names, max_display=10, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values, X_test_transformed, feature_names = ftr_names, max_display=10, plot_type="bar")

In [ ]:
index = 42 # the index of the point to explain
#print(explainer.expected_value[0]) # we explain class 0 predictions

shap.force_plot(explainer.expected_value[0], shap_values[0][index,:], 
                features = X_test_transformed[index,:],feature_names = ftr_names,show=True)

#plt.savefig('XGBoost SHAP Local Feature Importance Class 0', dpi=1200)
#plt.show()

In [ ]:
index = 42 # the index of the point to explain
print(explainer.expected_value[1]) # we explain class 0 predictions

shap.force_plot(explainer.expected_value[1], shap_values[1][index,:], 
                features = X_test_transformed[index,:],feature_names = ftr_names,show=True)

In [ ]:
index = 42 # the index of the point to explain
print(explainer.expected_value[2]) # we explain class 0 predictions

shap.force_plot(explainer.expected_value[2], shap_values[2][index,:], features = X_test_transformed[index,:],
                feature_names = ftr_names,show=True)